## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2021-10-02 15:39:10,-34.5322,20.0403,56.28,50,90,13.80
1,1,Mao,ES,2021-10-02 15:36:46,39.8885,4.2658,73.96,68,40,8.05
2,2,Aksarka,RU,2021-10-02 15:39:11,66.5606,67.7975,31.01,81,76,11.43
3,3,Nirasaki,JP,2021-10-02 15:39:11,35.7000,138.4500,61.05,94,1,2.86
4,4,Katangli,RU,2021-10-02 15:39:12,51.7102,143.2326,42.98,70,6,8.88


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df



,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Mao,ES,2021-10-02 15:36:46,39.8885,4.2658,73.96,68,40,8.05
6,6,Chama,CH,2021-10-02 15:39:12,47.1821,8.4636,71.76,56,70,2.30
7,7,Rikitea,PF,2021-10-02 15:39:12,-23.1203,-134.9692,72.64,72,2,21.25
11,11,Puerto Del Rosario,ES,2021-10-02 15:39:14,28.5004,-13.8627,83.88,78,0,17.27
13,13,Shelburne,CA,2021-10-02 15:39:14,44.0787,-80.2041,70.12,61,99,13.67
...,...,...,...,...,...,...,...,...,...,...
673,673,San Nicolas,PH,2021-10-02 15:42:24,18.1725,120.5953,78.71,83,20,2.30
675,675,Manggar,ID,2021-10-02 15:42:24,-2.8833,108.2667,79.12,82,96,3.62
677,677,Camacha,PT,2021-10-02 15:42:25,33.0833,-16.3333,72.36,64,40,12.66
678,678,Bergama,TR,2021-10-02 15:42:25,39.1207,27.1805,76.23,68,9,15.05


In [17]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()


City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace=True)

C:\Users\pkdwy\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

KeyError: "['Current Description'] not in index"

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

clean_hotel_df = hotel_df.copy()
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(inplace=True )
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig